### LH Model

In [4]:
import psycopg2
from datetime import datetime, timedelta, date
import datetime
import copy
import numpy as np
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from df2gspread import df2gspread as d2g
from tqdm import tqdm
from pulp import *
import math
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
import googlemaps
import time
import warnings
warnings.filterwarnings('ignore')
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

#### Constructing dataframes for LH

In [105]:
child_whs = pd.read_csv('child_whs.csv')
parent_whs = pd.read_csv('parent_whs.csv')

In [122]:
child_whs.head(2)

,Unnamed: 0,created_at,hour,activation,wh_id,wh,product_id,stock_minunit,basic_unit_count,stock,...,rr_minunit,avg_price,wh_cont,total_dc_stock,rr_max,base,gap,n_rop,rop,doh
0,0,2022-06-14,21.0,False,1,Mostorod,2,0.0,1.0,0.0,...,0.0,96.0,0.15248,0.0,0.0,2.5,1.5,0.0,0.0,1.5
1,1,2022-06-14,21.0,False,1,Mostorod,4,0.0,1.0,0.0,...,0.0,151.0,0.15248,0.0,0.0,2.5,1.5,0.0,0.0,1.5


In [123]:
parent_whs.head(2)

,Unnamed: 0,created_at,hour,activation,wh_id,wh,product_id,stock_minunit,basic_unit_count,stock
0,0,2022-06-14,21.0,False,4,Mostorod DC,2,0.0,1.0,0.0
1,1,2022-06-14,21.0,False,4,Mostorod DC,4,0.0,1.0,0.0


In [106]:
rep = child_whs.copy()

In [107]:
stock = parent_whs.copy()

In [108]:
rep['current_doh'] = rep['stock']/rep['rr']

In [109]:
rep = rep[rep.current_doh <= rep.rop]

In [110]:
rep['target'] = rep['doh']*rep['rr']  
rep.target = rep.target.apply(math.ceil)

In [111]:
rep['demand'] = rep['target'] - rep['stock'] 

#### Optimization FX

In [115]:
rep_df = rep.loc[rep['wh'].isin(FC_cairo)].copy()
rep_df = rep_df.loc[~(rep_df['demand'] == np.inf)]

stock_df = stock.loc[stock['wh'].isin(DC_cairo)].copy()

In [116]:
stock_df = stock_df.drop_duplicates() 
rep_df = rep_df.drop_duplicates()

In [117]:
lh_error = []
main_lh = pd.DataFrame()
for id in tqdm(rep_df.product_id.unique()):
    try:
        stock_test = stock_df[(stock_df.product_id == id)].reset_index()[['product_id','wh_id','wh','stock']].copy()
        rep_test = rep_df[(rep_df.product_id == id)].reset_index()[['product_id','wh_id','wh','stock','target','demand','avg_price','rr']].copy()
        stock_test.wh = stock_test.wh.str.replace('-','_')
        stock_test.wh = stock_test.wh.str.replace(' ','_')
        rep_test.wh = rep_test.wh.str.replace('-','_')
        rep_test.wh = rep_test.wh.str.replace(' ','_')

        model = LpProblem("Minimize_LR", LpMinimize)
        cust = list(rep_test['wh'])
        warehouse = list(stock_test['wh'])

        stock_dict = dict(stock_test[['wh','stock']].values)
        demand = dict(rep_test[['wh','demand']].values)

        price = dict(rep_test[['wh','avg_price']].values)
        target = dict(rep_test[['wh','target']].values)
        available_stock = dict(rep_test[['wh','stock']].values)
        rr = dict(rep_test[['wh','rr']].values)

        rep_test['total_demand'] = rep_test.demand.sum()
        rep_test['demand_cont'] = rep_test['demand']/(rep_test['total_demand']+.00000000000001)
        demand_cont = dict(rep_test[['wh','demand_cont']].values)

        x = LpVariable.dicts("Move_", [(w,c) for w in warehouse for c in cust], lowBound=0, cat="Integer")
        model += lpSum(([(((((demand[c]-x[(w,c)]) * price[c]))))  for w in warehouse for c in cust]))
        

        for c in cust:
            model += lpSum(x[(w,c)] for w in warehouse) <= demand[c]
        for w in warehouse:
            model += lpSum(x[(w,c)] for c in cust) <= stock_dict[w]

        if (sum(stock_dict.values())) < sum(demand.values()):
            for c in cust:
                model += lpSum(x[(w,c)] for w in warehouse) <= (demand_cont[c]*sum(stock_dict.values()))


        # Solve Model & Produce output in a dataframe
        solver = PULP_CBC_CMD(timeLimit=150, msg=0)
        model.solve(solver)

        # print("Status:", LpStatus[model.status])

        lh = [{'wh_from':"{}".format(w),'wh_to':"{}".format(c), 'quant':x[(w,c)].varValue} for w in warehouse for c in cust]
        lh = pd.DataFrame(lh)
        lh['product_id'] = id

        main_lh = main_lh.append(lh)
    except:
        lh_error.append(id)
main_lh.reset_index(drop = True)
main_lh = main_lh[['product_id','wh_from','wh_to','quant']]

100%|██████████| 1890/1890 [01:23<00:00, 22.74it/s]


In [118]:
main_lh = main_lh[main_lh.quant != 0].reset_index().drop(columns='index')

In [119]:
main_lh.head(10)

,product_id,wh_from,wh_to,quant
0,191,Barageel,Mostorod,28.0
1,191,Barageel,Abu_Rawash,9.0
2,191,Barageel,El_Marg,20.0
3,191,Barageel,Basatin,29.0
4,191,Barageel,Basous,6.0
5,191,Barageel,Cluster_Haram_warehouse,3.0
6,233,Anshas,Mostorod,1.0
7,233,Anshas,Abu_Rawash,1.0
8,233,Anshas,El_Marg,1.0
9,233,Anshas,Basatin,1.0
